# Thermosolutal convection of a Darcy fluid

In [ ]:
from lucifex.fdm import AB2, CN
from lucifex.sim import run, xdmf_to_npz
from lucifex.viz import plot_colormap_multifigure, create_animation, save_figure, display_animation, get_ipynb_file_name
from crocodil.dns import dns_darcy_thermosolutal

STORE = 1
WRITE = None
DIR_BASE = f'./data_{get_ipynb_file_name()}'
DIR_PARAMS = ('Ra', 'Le', 'gamma', 'Nx', 'Ny')

simulation = dns_darcy_thermosolutal(
    store_delta=STORE, 
    write_delta=WRITE, 
    dir_base=DIR_BASE, 
    dir_params=DIR_PARAMS,
    dir_timestamp=True,
)(
    Nx=64,
    Ny=64,
    cell='quadrilateral', 
    scaling='advective',
    Ra=200.0,
    Le=2.0,
    gamma=1.0,
    D_adv_solutal=AB2, 
    D_diff_solutal=CN,
    D_adv_thermal=AB2, 
    D_diff_thermal=CN,
    diagnostic=True,
)

n_stop = 200
dt_init = 1e-3
n_init = 10
run(simulation, n_stop=n_stop, dt_init=dt_init, n_init=n_init)

if WRITE:
    xdmf_to_npz(simulation, delete_xdmf=False)

c, theta, u, psi = simulation['c', 'theta', 'u', 'psi']

In [3]:
title_doublet = lambda time_doublet, tex_doublet: (
    tuple(f'${n}(t={t:.6f})$' for t, n in zip(time_doublet, tex_doublet))
)
series_doublet = (c, theta)
tex_doublet = ('c', '\\theta')

time_slice = slice(0, None, 2)
time_doublet_series = [ti for ti in zip(*[s.time_series[time_slice] for s in series_doublet])]

anim = create_animation(
    plot_colormap_multifigure(n_cols=2, colorbar=False),
    cmaps=('magma', 'plasma'),
)(
    [ui for ui in zip(*[s.series[time_slice] for s in series_doublet])],
    titles=[title_doublet(i, tex_doublet) for i in time_doublet_series],
)
anim_path = save_figure('c(t)_theta(t)', simulation.dir_path, prefix_ipynb=False, get_path=True)(anim)

display_animation(anim_path)

CalledProcessError: Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '1920x720', '-pix_fmt', 'rgba', '-r', '10.0', '-loglevel', 'error', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-y', '/Users/George/Desktop/CRoCoDiL/demo/notebooks/benchmark/./data_04_darcy_thermosolutal/Ra=200.0|Le=2.0|gamma=1.0|Nx=64|Ny=64__26-01-29_01-00-03.63/c(t)_theta(t).mp4']' returned non-zero exit status 255.